MICRO-452 Basics of mobile robotics

Project Report



In [5]:
import numpy as np

### Table of Contents

* [Introduction](#introduction)
* [Filtering](#filtering)
    * [General, filtering](#genfiltering)
    * [Choosing a filter](#filterchoice)
    * [Mathematical model](#mathmodel)
    * [Filter initialization](#filterinit)
    * [Testing](#testing)
    * [Conclusion on filtering](#filterconcl)

### Introduction <a class="anchor" id="introduction"></a>
The present project takes place in the Basics of mobile robotics course. The goal is to programm a mobile Thymio robot to perform a chosen task. The programm must use computer vision, global navigation, local navigation and filtering to determine the robot's position and make it achieve its objectives.

We decided to programm Santo, an unfortunately blue santa that unfortunately has no gifts but is dedicated to its job of visiting all the houses it can.

The programm uses an overhead camera to detect the different obstacles, the houses and Santo's position. A global navigation module allows Santo to do some path planning to reach every target. Finally, a Kalman filter using the robot's position from the camera and odometry allows the Thymio to know and correct its position in real time.


### Filtering <a class="anchor" id="filtering"></a>
#### General Filtering <a class="anchor" id="genfiltering"></a>
The goal of having a filter in such a project is to take into account the different uncertainties when trying to localize the robot. To "localize" the robot means knowing its state, which we defined as its position, angle, velocity and angular velocity with respect to a given reference point.

There are multiple ways to gather information about the robot's position with the chosen configuration.
- Using the informations given by the overhead camera
- Using the odometry and the robot's speed
- Using the accelerometer

We chose to use the informations given by the overhead camera to derive th robot's position relative to the bottom left corner of the image and its angle relative to the x and y axis of said image.

#### Choosing a filter <a class="anchor" id="filterchoice"></a>
The chosen filter has to be bayesian. It also should be able to merge all sensors' data to compute the correct robot's position.
As seen during the course, there are multiple bayesian filters following the given criteria. The ones that we explored were: 

- A particle filter. It uses the environment's features to determine a probability distribution of the position of the robot. It requires good sensor's inputs.
- A Kalman filter. A mathematical model is derived to predict the robot's state. It is then updated using sensor's measurements.

We chose the Kalman filter as it seemed to be the simplest one. Placing the Thymio in a mainly 2D plane didn't allow us to use the infrared sensors for the particle filter. Moreover, the Kalman filter felt more interesting as it is applied in many domains to which students are often confronted.

#### Mathematical model <a class="anchor" id="mathmodel"></a>
The state represents the informations we want to know about the Thymio.

$$\hat{\mu} = 
\begin{bmatrix}
x\\
y\\
\dot{x}\\
\dot{y}\\
\theta\\
\dot{\theta}\\
\end{bmatrix}
= 
\begin{bmatrix}
\mu_{0}\\
\mu_{1}\\
\mu_{2}\\
\mu_{3}\\
\mu_{4}\\
\mu_{5}\\
\end{bmatrix}$$

Where:
- x is the x-coordinate of the robot relative to the bottom left corner of the camera image
- y is the y-coordinate of the robot relative to the bottom left corner of the camera image
- $\dot{x}$ is the speed of the robot in the x direction
- $\dot{y}$ is the speed of the robot in the y direction
- $\theta$ is the robot's angle relative to the image's x and y axis
- $\dot{\theta}$ is the angular velocity of the robot

The numbers $\mu_{0}$ to $\mu_{5}$ were attributed to ease the implementation in the code.

![Thymio](Thymio.png)

The robot receive commands for the left and right motors from the computer. Those two variables are therefore chosen as inputs in the Kalman filter:

$$u = \begin{bmatrix}
u_{1}\\
u_{2}\\
\end{bmatrix}
$$

After choosing the state variables, it is possible to define the mathematical model for the filter.
The model is not linear so an extended Kalman filter is used. It's general representation is given as follows:

$$
\begin{align}
    \left. \begin{array}{l}
        \bar{\mu_{t}} = g(u_{t}, \mu_{t-1})\\
        \bar{\Sigma_{t}} = G_{t} \Sigma_{t-1} G_{t} + R_{t}\\
    \end{array}\right\} Prediction\, step\\
    \\
    \left. \begin{array}{l}
        K_{t} = \bar{\Sigma_{t}} H_{t}^T (H_{t} \bar{\Sigma_{t}} H_{t}^T + Q_{t})^{-1}\\
    \end{array}\right\} Gain\, computation\\
    \\
    \left. \begin{array}{l}
        \mu_{t} = \bar{\mu_{t}} + K_{t} (y_{t} - h(\mu_{t})\\
        \Sigma_{t} = (I - K_{t} H_{t}) \bar{\Sigma_{t}}\\
    \end{array}\right\} Update\, step
\end{align}
$$

The motion model $g$ is purely base on the robot's state and on the inputs.

$$g(u_{t}, \mu_{t-1}) = 
\begin{bmatrix}
\mu_{0} + \frac{T_{1}}{2} \cos(\mu_{4}) [u_{1} + u_{2}]\\
\mu_{1} + \frac{T_{1}}{2} \sin(\mu_{4}) [u_{1} + u_{2}]\\
\frac{1}{2} \cos(\mu_{4}) [u_{1} + u_{2}]\\
\frac{1}{2} \sin(\mu_{4}) [u_{1} + u_{2}]\\
\mu_{4} + \frac{T_{1}}{2r} [u_{1} - u_{2}]\\
\frac{1}{2r} [u_{1} - u_{2}]\\
\end{bmatrix}$$

With $T_{1}$ the time interval between two iterations of the filter and $r$ the radius of the Thymio robot.



In [8]:
def motModel(x, u, T1, r):
    g = np.array([0., 0., 0., 0., 0., 0.]) #just initializing a 2D array
    g[0] = (x[0] + T1*math.cos(x[4])*(u[0] + u[1])/2)
    g[1] = x[1] + T1*math.sin(x[4])*(u[0] + u[1])/2
    g[2] = math.cos(x[4])*(u[0] + u[1])/2
    g[3] = math.sin(x[4])*(u[0] + u[1])/2
    g[4] = x[4] + T1*(u[0] - u[1]) / (2*r)
    g[5] = (u[0] - u[1]) / (2*r)
    return 

It is possible to linearize this function using Taylor's expansion:

$$f(x,y) \approx {f(a,b) + \frac{\partial f(x,y)}{\partial x}\bigg|_{(a,b)}(x-a) + \frac{\partial f(x,y)}{\partial y}\bigg|_{(a,b)}(y-b)}$$

Computing the jacobian, we get the matrix $G$:

$$G = 
\begin{bmatrix}
1 & 0 & 0 & 0 & -\frac{T_{1}}{2}\sin(u_{1}+u_{2}) & 0\\
0 & 1 & 0 & 0 & \frac{T_{1}}{2}\cos(u_{1}+u_{2}) & 0\\
0 & 0 & 0 & 0 & -\frac{1}{2}\sin(u_{1}+u_{2}) & 0\\
0 & 0 & 0 & 0 & \frac{1}{2}\cos(u_{1}+u_{2}) & 0\\
0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
\end{bmatrix}$$



In [9]:
def Gjacobian(theta, m1 ,m2, T1, r):
    G = np.array([[0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.],
                  [0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.]])
    G[0,0] = 1
    G[1,1] = 1
    G[0,4] = -T1*math.sin(theta)*(m1+m2)/2
    G[1,4] = T1*math.cos(theta)*(m1+m2)/2
    G[2,4] = -math.sin(theta)*(m1+m2)/2
    G[3,4] = math.cos(theta)*(m1+m2)/2
    G[4,4] = 1
    return G

The measurement model represents the relation between the measured values and the state model of the robot. For simplicity, our group decided to use only the measurements of the camera to derive the x and y position of the robot as well as its angle. 

$$y_{t}(\mu_{t}) = 
\begin{bmatrix}
camX\\
camY\\
cam\theta\\
\end{bmatrix}$$

The measurement model $h$ is linear and is directly given:

$$h(\mu_{t}) = y_{t}(\mu_{t})$$

The matrix $H$ is the jacobian of $h$:

$$H = 
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0\\
\end{bmatrix}$$

In [10]:
def measModel(x):
    h = np.array([0.,0.,0.])
    h[0] = x[0]
    h[1] = x[1]
    h[2] = x[4]
    return h

def Hjacobian(theta):
    H = np.array([[0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.]])
    H[0,0] = 1
    H[1,1] = 1
    H[2,4] = 1
    return H

The camera outputs an x = 0 and y = 0 position for the robot when it is hidden. This induced errors in the state estimation. To mitigate this problem, it was chosen not to take measurements into account when the camera doesn't detect any Thymio. This is done by setting the measurement model $h = 0$ and its jacobian $H = 0$.

In [11]:
def measModel(x,camstate):
    h = np.array([0.,0.,0.])
    h[0] = x[0]
    h[1] = x[1]
    h[2] = x[4]
    if camState == False:
        h[0] = 0
        h[1] = 0
        h[2] = 0
    return h

def Hjacobian(theta, camState):
    H = np.array([[0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.]])
    H[0,0] = 1
    H[1,1] = 1
    H[2,4] = 1
    if camState == False:
        H[0,0] = 0
        H[1,1] = 0
        H[2,4] = 0
    return H

#### Filter's initialization <a class="anchor" id="filterinit"></a>
Upon starting the programm, multiple values must be initialized in order for the Kalman filter to perform correctly.

The first one is the speed conversion constant.

In [2]:
#Convert speed commands in mm/s
speed_conv = 0.36

This value depends on the robot used for the experiment. It might also vary depending on the surface the Thymio is rolling on as it might slip during its movements. It is found experimentally by doing some calibration steps.

The next value to initalize is the Thymio's radius, or more precisely the middle distance between the two wheels.

In [3]:
#Thymio's radius
r = 47 #mm

The incertitude matrices $R$ and $Q$ are found experimentally. They are tuned doing testing so the robot's state estimation is as precise as possible. In our case, the tuned matrices are given below:

In [6]:
#uncertainty on state
R = np.array([[0.01,0.,0.,0.,0.,0.],[0.,0.01,0.,0.,0.,0.],[0.,0.,0.01,0.,0.,0.],[0.,0.,0.,0.01,0.,0.],
              [0.,0.,0.,0.,0.00001,0.],[0.,0.,0.,0.,0.,0.1]])
#uncertainty on measurement
Q = np.array([[0.01,0.,0.],[0.,0.01,0.],[0.,0.,0.01]])

Finally, the Thymio's first state initialization is done using the first frame of the camera when the programm is launched. It allows the Kalman to start with a relatively precise position and angle for the robot.

#### Testing <a class="anchor" id="testing"></a>
The Kalman filter was first tested using simulated data, meaning the fake measures representing the Thymio going in straight lines with sharp turn were inputed in the filter.

The second testing method was not relying on the camera measures as the camera setup wasn't available yet. The Thymio's movements were solely based on the motion model $g$.

Then, tests were made on the final setup. It allowed to tune the incertitude matrices and the speed conversion constant.

#### Conclusion on filtering <a class="anchor" id="filterconcl"></a>
The extended Kalman filter satifies our needs for this project. It allows to merge the camera measurements with a model of the robot's motion using the motors' commands to compute an estimation of the Thymio's position, speed and angle.